In [6]:
import math

def datetime_to_jday_utsec(dt):
    """
    Converts a Python datetime object (UTC) to Julian Day and UT seconds for Geopack.

    Args:
        dt (datetime.datetime): The datetime object (assumed to be in UTC).

    Returns:
        tuple: (jday, utsec)
               jday (float): Julian Day (starts at noon UT).
                             Note: Geopack often uses a definition where jday
                             represents the start of the UT day (00:00).
                             This function calculates the standard astronomical Julian Date.
                             Let's test if geopack handles this standard definition.
               utsec (float): Seconds past midnight UT for the given day.
    """
    # Calculate seconds past midnight UT
    utsec = dt.hour * 3600.0 + dt.minute * 60.0 + dt.second + dt.microsecond / 1e6

    # Calculate Julian Day Number for 0h UT using standard algorithm
    # (e.g., Meeus, Astronomical Algorithms, Chapter 7)
    year = dt.year
    month = dt.month
    day = dt.day

    if month <= 2:
        year -= 1
        month += 12

    a = math.floor(year / 100.0)
    b = 2 - a + math.floor(a / 4.0)
    jd_0h = math.floor(365.25 * (year + 4716)) + math.floor(30.6001 * (month + 1)) + day + b - 1524.5

    # Add the fraction of the day corresponding to utsec
    # The standard JD starts at Noon, so 0h UT is JD - 0.5
    # However, geopack functions likely expect the 'day number' part and the 'time' part separately.
    # Let's provide the JD calculated for 0h UT and the separate utsec.
    # Often, the integer part of JD for 0h UT is used as the 'day number'.
    # Let's try returning the full JD for 0h + fractional day.
    jday = jd_0h + utsec / 86400.0

    # Alternative: Some conventions might just use the JD for 0h UT as 'jday' parameter
    # jday_alt = jd_0h # Let's stick to the full JD first.

    # Recheck geopack common usage: Often jday is treated as the day count since an epoch.
    # Let's use the simpler toordinal approach which might be sufficient for geopack,
    # although it's not the *true* Julian date.
    # Ref: datetime.toordinal() returns days since 0001-01-01 (Proleptic Gregorian)
    # JD Epoch (Noon, Jan 1, 4713 BC) corresponds to Gregorian date Nov 24, 4714 BC.
    # J2000.0 = JD 2451545.0 = 2000-01-01 12:00 UT
    # Ordinal for 2000-01-01 is 730120.
    # Ordinal for 0001-01-01 is 1.
    # Let's calculate JD properly as above, it's safer.

    # Return the Julian date including the fraction of the day
    return jday, utsec

In [7]:
import numpy as np
import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import geopack

# --- 1. 设置时间和太阳风参数 ---
# 时间
dt = datetime.datetime(2005, 1, 1, 0, 0, 0)
jday, utsec = datetime_to_jday_utsec(dt)
print(f"Time: {dt}, Julian Day: {jday}, UT Seconds: {utsec}") # Now jday will be a float

# 太阳风和地磁活动参数 (示例值)
parmod = np.zeros(10)
parmod[0] = 2.0  # Solar wind dynamic pressure (nPa)
parmod[1] = -20.0 # Dst index (nT)
parmod[2] = 0.0   # IMF By (nT) - GSM
parmod[3] = -2.0  # IMF Bz (nT) - GSM

# --- 2. 定义地理起始点并转换为 GSM ---
# 地理坐标
lat_start_deg = 40.0   # 纬度 (北纬为正)
lon_start_deg = 116.0  # 经度 (东经为正)
alt_start_km = 500.0   # 海拔高度 (km)

print(f"Starting point (Geographic): Lat={lat_start_deg:.2f} deg, Lon={lon_start_deg:.2f} deg, Alt={alt_start_km:.1f} km")

# 步骤 2a: Geographic -> GEO (Cartesian)
xgeo, ygeo, zgeo = geopack.geodgeo(lat_start_deg, lon_start_deg, alt_start_km)
print(f"  -> Intermediate GEO (Cartesian): X={xgeo:.3f}, Y={ygeo:.3f}, Z={zgeo:.3f} RE")

# 步骤 2b: GEO -> GSM
x_start_gsm, y_start_gsm, z_start_gsm = geopack.geogsm(xgeo, ygeo, zgeo, jday, utsec)
print(f"  -> Converted Starting point (GSM): X={x_start_gsm:.3f}, Y={y_start_gsm:.3f}, Z={z_start_gsm:.3f} RE")

# --- 3. 设置追踪参数 ---
ds = 0.1       # 步长 (RE)
rlim = 1.0      # 停止追踪的径向距离 (RE) - 地球表面附近
maxloop = 10000 # 最大步数

# 模型选择
exopt = geopack.T96  # 外部场模型
iopt = geopack.IGRF   # 内部场模型

# --- 4. 执行磁力线追踪 (使用转换后的 GSM 坐标) ---

# 追踪到北半球 (dir=1)
print("\nTracing towards Northern Hemisphere...")
x_n, y_n, z_n, _, _, _, _, r_n = geopack.trace(
    x_start_gsm, y_start_gsm, z_start_gsm, # 起始点 GSM (RE)
    jday, utsec, dir=1, ds=ds, rlim=rlim, maxloop=maxloop,
    parmod=parmod, exopt=exopt, iopt=iopt
)
print(f"  -> Finished tracing North. Number of points: {len(x_n)}")
if len(x_n) > 0:
    print(f"  -> Northern footpoint (GSM approx): X={x_n[-1]:.2f}, Y={y_n[-1]:.2f}, Z={z_n[-1]:.2f} RE, R={r_n[-1]:.2f} RE")
else:
    # 如果起始点在北半球，向北追踪可能立即停止或步数很少
    print("  -> Trace might have stopped immediately (already near Northern footpoint?).")

# 追踪到南半球 (dir=-1)
print("Tracing towards Southern Hemisphere...")
x_s, y_s, z_s, _, _, _, _, r_s = geopack.trace(
    x_start_gsm, y_start_gsm, z_start_gsm, # 起始点 GSM (RE)
    jday, utsec, dir=-1, ds=ds, rlim=rlim, maxloop=maxloop,
    parmod=parmod, exopt=exopt, iopt=iopt
)
print(f"  -> Finished tracing South. Number of points: {len(x_s)}")
if len(x_s) > 0:
    print(f"  -> Southern footpoint (GSM approx): X={x_s[-1]:.2f}, Y={y_s[-1]:.2f}, Z={z_s[-1]:.2f} RE, R={r_s[-1]:.2f} RE")
else:
     # 如果起始点在南半球，向南追踪可能立即停止或步数很少
    print("  -> Trace might have stopped immediately (already near Southern footpoint?).")


# --- 5. 合并和可视化 (仍然在 GSM 坐标系中) ---
# 合并两个方向的路径点（南半球的需要反转顺序）
# 确保起始点只包含一次
x_trace = np.concatenate((x_s[::-1], [x_start_gsm], x_n))
y_trace = np.concatenate((y_s[::-1], [y_start_gsm], y_n))
z_trace = np.concatenate((z_s[::-1], [z_start_gsm], z_n))

# 3D 可视化
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# 绘制磁力线
ax.plot(x_trace, y_trace, z_trace, label='Traced Field Line', color='blue')

# 绘制起始点 (GSM 坐标)
ax.scatter([x_start_gsm], [y_start_gsm], [z_start_gsm], color='red', s=50, label=f'Start Point (GSM)')

# 绘制地球
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
earth_x = 1 * np.outer(np.cos(u), np.sin(v))
earth_y = 1 * np.outer(np.sin(u), np.sin(v))
earth_z = 1 * np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_surface(earth_x, earth_y, earth_z, color='lightblue', alpha=0.6, rstride=4, cstride=4)

# 设置坐标轴标签和范围 (GSM)
ax.set_xlabel('X GSM (RE)')
ax.set_ylabel('Y GSM (RE)')
ax.set_zlabel('Z GSM (RE)')
# 自动调整范围可能更好
coords = np.vstack([x_trace, y_trace, z_trace]).T
center = coords.mean(axis=0)
max_range = np.max(np.sqrt(np.sum((coords - center)**2, axis=1))) * 1.2
max_range = max(max_range, 2.5) # 保证最小范围包含地球
ax.set_xlim([center[0] - max_range, center[0] + max_range])
ax.set_ylim([center[1] - max_range, center[1] + max_range])
ax.set_zlim([center[2] - max_range, center[2] + max_range])

# 添加标题和图例
ax.set_title(f'Magnetic Field Line Trace (T96+IGRF)\nStart: {lat_start_deg:.1f}N, {lon_start_deg:.1f}E, {alt_start_km:.0f}km\n{dt} UT')
ax.legend()

# 调整视角
ax.view_init(elev=30., azim=-45)

plt.show()

Time: 2005-01-01 00:00:00, Julian Day: 2453371.5, UT Seconds: 0.0
Starting point (Geographic): Lat=40.00 deg, Lon=116.00 deg, Alt=500.0 km


AttributeError: module 'geopack' has no attribute 'geodgeo'

In [8]:
from geopack import geopack
import numpy as np

In [9]:
# 设置时间
t = geopack.recalc(2023, 1, 1, 0, 0)

TypeError: recalc() takes from 1 to 4 positional arguments but 5 were given